<a href="https://colab.research.google.com/github/Muhammad-Usama-07/ComputerVision/blob/master/ComputerVisionWorkSolution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Save all images of video 

In [ ]:
import cv2
camera = cv2.VideoCapture("NewObjectDetector.mp4")
face_cascade = cv2.CascadeClassifier('haarcascades/haarcascade_frontalface_default.xml')
counter = 0

while camera.isOpened():
    ret, frame = camera.read()
    if ret:
        
        try:
            image = cv2.resize(frame, (960, 540))
            print(f'Saving Image No. {counter}')
            cv2.imwrite(f'Data/{counter}.bmp', image)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
        except:
            pass
        counter +=1
    else:
        print('Frame not available')
        break
